#### モデルフリーの強化学習
１．Q-Learning\
    Q-learningはTD学習の中でも方策オフ型の手法です。方策はε-greedyなりsoftmaxなりを使いますので、方策を改善することはありません。


##### Q-Learningの学習フロー
１．Q関数の初期化、方策の決定、初期状態の決定\
２．方策に従って報酬、次の状態の観測\
３．TD Errorの計算\
$\delta_t:=r_t+\gamma \max _{a^{\prime} \in \mathcal{A}} \hat{Q}\left(s_{t+1}, a^{\prime}\right)-\hat{Q}\left(s_t, a_t\right)$\
４．Q関数の更新\
$\hat{Q}\left(s_t, a_t\right):=\hat{Q}\left(s_t, a_t\right)+\alpha_t \delta_t$
        
##### Q-LearningをJaxと行列形式で解いていく
すごい人の方法をまねて実装していきましょう!()

**Q-Learning**
* 近似ベルマン作用素: $\hat{T}Q(s, a) = r(s, a) + \gamma \max_{a'} Q(s', a')$
* 更新則: $Q_{t+1}(s_t, a_t) \leftarrow Q_{t}(s_t, a_t) + \alpha_t \left(\hat{T}Q_t(s_t, a_t) - Q_t(s_t, a_t)\right)$

上をまとめると、タイムステップごとに近似ベルマン作用素を計算し、それを用いてQの値をこうしんします。

* 更新則: $Q_{t+1} \leftarrow Q_t + \alpha_t D_t \left(\hat{T} Q_t - Q_t\right)$

**なぜこのようになるの？**



In [1]:
#Q Learningの実装

import random
from functools import partial
from typing import NamedTuple, Optional
import jax
import numpy as np
import jax.numpy as jnp
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
#RLベンチマーク
class MDP(NamedTuple):
    S: np.array
    A: np.array
    gamma : float
    horizon :int
    rew: float
    P: float
    phi: Optional[np.array] = None # 特徴行列
    optimal_Q: Optional[np.ndarray] = None  # 最適Q値
    optimal_Q_idx: Optional[np.ndarray] = None  # 後でplot用に使います   

    @property
    def State(self) -> int:
        return len(self.S)
    
    def Action(self) -> int:
        return len(self.A)
    

#貪欲方策　2.で使う

@jax.jit
def greedy_policy(Q: np.ndarray):
    greedy_policy = jnp.zeros_like(Q)

    S,A = Q.shape
    #Sの列にいき、その中で最もQの高いaを選択する
    greedy_policy = greedy_policy.at[jnp.arange(S),Q.argmax(axis=1)].set(1)
    assert greedy_policy.shape == (S,A)
    return greedy_policy

Q = np.zeros((1,3))


greedy_policy(Q)


Array([[1., 0., 0.]], dtype=float32)